In [1]:
# !pip install polars

In [2]:
#Some gdrive admin
# from google.colab import drive
# drive.mount('/content/drive')
#imports for data processing
#reading csv files
#data_path = "drive/My Drive/Data/"

#general house keep and loading the data

import polars as pl
import numpy as np

df = pl.read_csv("rawtrainers2.csv", null_values= ["Enter an answer","-99"])

In [3]:
df.columns

['RespondentID',
 'CollectorID',
 'Completion Date',
 'ExternalId',
 'Country of Residence',
 'Average Daily Step Count RC',
 'Average Daily Step Count',
 'Average Weekly Step Count RC',
 'Average Weekly Step Count',
 'Running Frequency',
 '50 Apps Installed on Smartphone-Banking',
 '50 Apps Installed on Smartphone-Revoult bank',
 '50 Apps Installed on Smartphone-Starling bank',
 '50 Apps Installed on Smartphone-Tandem bank',
 '50 Apps Installed on Smartphone-Credit and Online payments',
 '50 Apps Installed on Smartphone-Klarna',
 '50 Apps Installed on Smartphone-Fitness or sports',
 '50 Apps Installed on Smartphone-Fitbit',
 '50 Apps Installed on Smartphone-Adidas Running',
 '50 Apps Installed on Smartphone-Strava',
 '50 Apps Installed on Smartphone-My fitness pall',
 '50 Apps Installed on Smartphone-Peleton',
 '50 Apps Installed on Smartphone-Food and drink',
 '50 Apps Installed on Smartphone-Deliveroo',
 '50 Apps Installed on Smartphone-Mcdonalds',
 '50 Apps Installed on Smartphone-

In [4]:
#can I spot runners?
#the main select to create the dataframe
train_cols = [(col,col.split("?")[1].strip()) for col in df.columns if col[:2]=="Q7"]
df_running = df.select([pl.col('Average Weekly Step Count').cast(pl.UInt32).alias("weekstep"),
                        pl.col('Running Frequency').cast(pl.UInt32).alias("freq"),
                        pl.col('Gender').cast(pl.UInt8).alias("gender"),
                        pl.col('Age').cast(pl.UInt8).alias("age"),
                        pl.col('50 Apps Installed on Smartphone-Fitness or sports').cast(pl.Boolean).alias("sport"),
                        pl.col([
 'Openness',
 'Conscientious',
 'Agreeable',
 'Neurotic',
 'Extraverted',
 'Thrill-Seeking'])]+
                        [pl.col(t_in).alias(t_out).cast(pl.Boolean) for t_in,t_out in train_cols]
                       )
#sort functionality
df_running = df_running.sort("weekstep")
#main summariser
df_running.describe()

describe,weekstep,freq,gender,age,sport,Openness,Conscientious,Agreeable,Neurotic,Extraverted,Thrill-Seeking,Nike,adidas,Converse,Vans,Puma,Reebok,New Balance,Fila,Asics,Under Armour,Jordan,Timberland,Skechers,K-Swiss,Lacoste,Lotto,Mizuno,Salomon,Umbro,ecco,Diadora,Other,None of the above,Not applicable/prefer not to say,I don't know
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0,839.0
"""null_count""",1.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",27272.867542,4.756853,1.613826,35.282479,0.395709,58.353293,59.640719,69.962575,54.258982,45.269461,45.943677,0.656734,0.578069,0.337306,0.290822,0.194279,0.139452,0.156138,0.073897,0.082241,0.087008,0.076281,0.057211,0.178784,0.015495,0.047676,0.007151,0.007151,0.026222,0.023838,0.011919,0.016687,0.137068,0.033373,0.009535,0.007151
"""std""",23018.151813,1.593692,0.487162,8.812151,0.489294,14.340055,19.464842,19.173097,21.785706,23.017594,20.685762,0.475082,0.494162,0.473072,0.454413,0.395881,0.346624,0.363203,0.26176,0.274895,0.282015,0.265606,0.232384,0.3834,0.123583,0.213206,0.084313,0.084313,0.159889,0.152635,0.108586,0.12817,0.344124,0.179716,0.097239,0.084313
"""min""",0.0,1.0,1.0,18.0,0.0,6.25,6.25,6.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""max""",143215.0,6.0,2.0,55.0,1.0,93.75,100.0,100.0,100.0,100.0,100.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""median""",23954.0,6.0,2.0,35.0,0.0,62.5,62.5,68.75,56.25,43.75,47.22,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",8231.0,3.0,1.0,29.0,null,50.0,43.75,56.25,37.5,25.0,31.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""75%""",42110.0,6.0,2.0,42.0,null,68.75,75.0,87.5,68.75,62.5,61.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [5]:
#can use an array index
df_running[1,1]

1

In [6]:
#I can never remember value_counts or count_values!
df_running["gender"].value_counts()

gender,counts
u8,u32
1,324
2,515


In [7]:
#applying a function using (wait for it...) APPLY
def level(x):
  if 33 < x < 66:
    return 2
  elif x < 33:
    return 1
  elif x > 66:
    return 3 
    
df_running = df_running.with_columns([pl.col("Thrill-Seeking").apply(level).alias("thrill"),
                                      pl.col('Conscientious').apply(level).alias("con"),
                                      pl.col('Extraverted').apply(level).alias("extra")])

df_running.groupby("thrill").agg([pl.col("weekstep").mean()])

thrill,weekstep
i64,f64
3,29429.816327
1,22710.175966
null,39734.75
2,28709.18


In [8]:
#do runner of different conscientiousness do different weekely running steps

df_running.groupby("con").agg([pl.col("weekstep").mean()])

con,weekstep
i64,f64
null,44960.0
2,27440.784543
1,22945.4875
3,27895.929664


In [9]:
#smae for extroverted people

df_running.groupby("extra").agg([pl.col("weekstep").mean()])

extra,weekstep
i64,f64
2,27763.916244
1,25879.90681
null,44960.0
3,28045.627329


In [10]:
#actually should we get rid of those at zero becas
df_running.filter(pl.col("weekstep")!=0).describe()

describe,weekstep,freq,gender,age,sport,Openness,Conscientious,Agreeable,Neurotic,Extraverted,Thrill-Seeking,Nike,adidas,Converse,Vans,Puma,Reebok,New Balance,Fila,Asics,Under Armour,Jordan,Timberland,Skechers,K-Swiss,Lacoste,Lotto,Mizuno,Salomon,Umbro,ecco,Diadora,Other,None of the above,Not applicable/prefer not to say,I don't know,thrill,con,extra
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0,701.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,4.0,4.0
"""mean""",32602.942939,4.814551,1.617689,34.947218,0.413695,58.473816,59.756098,70.220588,54.151722,45.588235,46.513702,0.673324,0.582026,0.350927,0.299572,0.188302,0.138374,0.166904,0.071327,0.088445,0.088445,0.078459,0.059914,0.172611,0.017118,0.042796,0.005706,0.005706,0.027104,0.021398,0.014265,0.017118,0.1398,0.029957,0.009986,0.007133,1.916306,2.302726,1.863702
"""std""",21436.120382,1.5436,0.486299,8.846876,0.492847,14.33966,19.344703,19.254194,22.081131,22.957496,20.422445,0.469333,0.493578,0.477601,0.458398,0.391233,0.345538,0.373157,0.257553,0.284144,0.284144,0.269085,0.237498,0.37818,0.129805,0.202541,0.075377,0.075377,0.162503,0.14481,0.118667,0.129805,0.347027,0.170591,0.099499,0.084214,0.66405,0.631005,0.717246
"""min""",433.0,1.0,1.0,18.0,0.0,6.25,6.25,6.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
"""max""",143215.0,6.0,2.0,55.0,1.0,93.75,100.0,100.0,100.0,100.0,100.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0
"""median""",29133.0,6.0,2.0,34.0,0.0,62.5,62.5,68.75,56.25,43.75,47.22,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0
"""25%""",16234.0,3.0,1.0,28.0,null,50.0,43.75,56.25,37.5,31.25,31.94,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,2.0,1.0
"""75%""",44669.0,6.0,2.0,42.0,null,68.75,75.0,87.5,68.75,62.5,61.11,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.0,3.0,2.0


In [11]:
#a little bit of bokeh
from bokeh.io import output_notebook, show
output_notebook()
from bokeh.plotting import figure
p = figure(width=400, height=400)
p.line(range(len(df_running["weekstep"])),df_running["weekstep"])
show(p) # show the results

Loading BokehJS ...

In [12]:
df_running = df_running.sort("freq")

In [13]:
p = figure(width=400, height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.line(range(len(df_running["freq"])),df_running["freq"])

show(p) # show the results

In [14]:
df_running.filter(pl.col("freq")!=6).describe()

describe,weekstep,freq,gender,age,sport,Openness,Conscientious,Agreeable,Neurotic,Extraverted,Thrill-Seeking,Nike,adidas,Converse,Vans,Puma,Reebok,New Balance,Fila,Asics,Under Armour,Jordan,Timberland,Skechers,K-Swiss,Lacoste,Lotto,Mizuno,Salomon,Umbro,ecco,Diadora,Other,None of the above,Not applicable/prefer not to say,I don't know,thrill,con,extra
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0,385.0
"""null_count""",1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0
"""mean""",28837.5625,3.290909,1.511688,34.023377,0.392208,57.356771,61.588542,69.287109,49.332682,48.193359,51.388438,0.735065,0.638961,0.32987,0.301299,0.231169,0.171429,0.202597,0.093506,0.114286,0.127273,0.114286,0.064935,0.148052,0.015584,0.072727,0.015584,0.012987,0.041558,0.044156,0.015584,0.025974,0.101299,0.007792,0.005195,0.005195,2.052219,2.359375,1.9296875
"""std""",23949.451325,1.249394,0.500514,8.689791,0.488878,14.360406,19.272616,19.415843,20.599647,22.380088,19.527778,0.441873,0.480927,0.470778,0.459419,0.422129,0.377373,0.402458,0.29152,0.318572,0.333712,0.318572,0.246732,0.355614,0.124022,0.260026,0.124022,0.113365,0.199838,0.205709,0.124022,0.159265,0.302116,0.088043,0.071981,0.071981,0.653135,0.605472,0.687641
"""min""",0.0,1.0,1.0,18.0,0.0,6.25,6.25,12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
"""max""",116885.0,5.0,2.0,54.0,1.0,93.75,100.0,100.0,100.0,100.0,100.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0
"""median""",25704.5,3.0,2.0,34.0,0.0,56.25,62.5,68.75,50.0,50.0,52.39,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0
"""25%""",8589.0,2.0,1.0,27.0,null,50.0,50.0,56.25,37.5,31.25,38.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.0,2.0,1.0
"""75%""",44042.0,4.0,2.0,40.0,null,68.75,75.0,87.5,62.5,62.5,65.28,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.0,3.0,2.0


In [15]:
p = figure(width=400, height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(df_running["weekstep"],df_running["freq"])

show(p) # show the results

In [16]:
p = figure(width=400, height=400)
df_running = df_running.sort("age")
# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(df_running["age"],df_running["weekstep"])

show(p) # show the results

In [17]:
#some groupbys 
df_running.groupby("gender").agg([pl.col("freq").mean(),pl.col("weekstep").mean()])

gender,freq,weekstep
u8,f64,f64
1,4.361111,29814.024691
2,5.005825,25671.048638


In [18]:
df_running.groupby("age").agg([pl.col("freq").mean(),pl.col("weekstep").mean()])

age,freq,weekstep
u8,f64,f64
18,3.666667,25844.333333
19,3.428571,24564.0
20,4.846154,29523.230769
21,3.75,16892.125
22,4.608696,30687.478261
23,4.6,24108.433333
24,5.185185,36652.962963
25,4.454545,26506.363636
26,4.36,29634.92


In [19]:
df_running.groupby("freq").agg([pl.col("weekstep").mean()])

freq,weekstep
u32,f64
2,29837.236111
4,30178.118644
5,28237.829787
6,25949.42511
3,30080.632812
1,20650.225806


In [20]:
df_running.groupby("sport").agg([pl.col("weekstep").mean()])

sport,weekstep
bool,f64
false,25228.507905
true,30388.668675


In [21]:
df_running.groupby("freq").agg([pl.col("sport").value_counts()])

freq,sport
u32,list[struct[2]]
4,"[{true,22}, {false,37}]"
1,"[{true,6}, {false,25}]"
3,"[{true,53}, {false,75}]"
2,"[{true,25}, {false,48}]"
5,"[{false,49}, {true,45}]"
6,"[{false,273}, {true,181}]"


In [22]:
#a quick look at the trainer brand data
train_cols = [(col,col.split("?")[1].strip()) for col in df.columns if col[:2]=="Q7"]
df_trainers = df.select([pl.col(t_in).alias(t_out).cast(pl.Boolean) for t_in,t_out in train_cols])
df_trainers.select([pl.col("*").value_counts()])


Nike,adidas,Converse,Vans,Puma,Reebok,New Balance,Fila,Asics,Under Armour,Jordan,Timberland,Skechers,K-Swiss,Lacoste,Lotto,Mizuno,Salomon,Umbro,ecco,Diadora,Other,None of the above,Not applicable/prefer not to say,I don't know
struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2]
"{false,288}","{false,354}","{false,556}","{true,244}","{false,676}","{false,722}","{false,708}","{true,62}","{true,69}","{true,73}","{false,775}","{false,791}","{true,150}","{false,826}","{false,799}","{true,6}","{true,6}","{true,22}","{false,819}","{false,829}","{true,14}","{false,724}","{false,811}","{false,831}","{false,833}"
"{true,551}","{true,485}","{true,283}","{false,595}","{true,163}","{true,117}","{true,131}","{false,777}","{false,770}","{false,766}","{true,64}","{true,48}","{false,689}","{true,13}","{true,40}","{false,833}","{false,833}","{false,817}","{true,20}","{true,10}","{false,825}","{true,115}","{true,28}","{true,8}","{true,6}"


In [23]:
#interacting with a series and producing rows of trainer
trainers = x = pl.Series("trainer", [[df_trainers.columns[i] for i,j in enumerate(k) if j] for k in df_trainers.iter_rows()])
df_trainers_shoe = df_running.hstack([trainers]).explode("trainer")
df_sum = df_trainers_shoe.groupby("trainer").agg([pl.col("freq").value_counts(),pl.col("weekstep").mean()])
df_sum

trainer,freq,weekstep
str,list[struct[2]],f64
"""Salomon""","[{2,2}, {6,12}, … {5,2}]",31619.727273
"""Puma""","[{1,2}, {6,94}, … {4,10}]",27818.846626
"""Asics""","[{1,4}, {6,34}, … {3,12}]",28363.985507
"""K-Swiss""","[{4,1}, {6,5}, … {5,2}]",45822.461538
"""Other""","[{3,11}, {2,10}, … {5,18}]",29088.313043
"""Timberland""","[{2,4}, {3,7}, … {5,5}]",25901.395833
"""Converse""","[{4,23}, {3,44}, … {2,22}]",28710.730496
"""Reebok""","[{5,14}, {2,13}, … {6,59}]",28657.179487
"""New Balance""","[{2,7}, {3,24}, … {1,6}]",27766.076336
